# Cali Pfleger
## 05/25/2024
- Model iCESM and CESM Last Millenium Ensemble 
- Climate System: Pacific Walker Circulation 
- Index: Sea Level Pressure Index (hPa)
- Variable: d18Op (0/00) (per mille)
- Time period: 1850-2005 
- Forcings: Full, GHG, LULC, OZA

# Calculations:
- Index Time Series (trend line, p-value, standered error (+1/-1) ribbon and ensmeble average)
- Climatological Average Maps
- Global Average Time Series (standered error ribbon and ensmeble average)
- Linear Trend Maps (95% CI significance stippling)- 
- Correlation with Index Maps (95% CI significance stippling)- 
- Effect of Forcing Maps (Correlation - PI Control Run Corrleation)
GCM Changes in Vertical Velocity (ω) Across the Tropical Pacific.
iCAM5 = 2xCO2 Experiment minus Modern Control, Meridional Mean (−20 to 20 latitude).Change in vertical velocity (ω) in the experiment minus the control in color filled contour plot. Black contour lines show control simulation. Sign convention is negative = upward motion, positive = downward motion. Thus, red colors in Western Pacific indicate a reduction in the magnitude of the negative values in the control compared to the experiment. Blue colors indicate less positive values in the experiment compared to the control, or less downward motion.

- Vertical Profile Maps (dD(per mille), omega, temperature (C), pressure (hPa))
- 

In [1]:
# Set Parameters and Naming 
model ='Model iCESM and CESM '
region = 'PWC Index ' # 'Eastern Pacific ' 'Western Pacific ' 
variable = 'SLP' #Sea Level Pressure
period = "Post Industrial"# "Full LME" #"Pre Industrial",
timep = "_post"# "_pre" # 
var_name = ['Full', 'GHG', 'LULC','OZA']#, 'ANTHRO']
timep_title = "Post Industrial"#'Pre Industrial' 

In [2]:
# Index Box Parameters 
## Make geom for E and W PWC index boxes
from shapely import geometry
from shapely.geometry.polygon import LinearRing, Polygon
Elats = [-5, -5, 5, 5]
Elons = [90, 150, 150,90]
Ering = LinearRing(list(zip(Elons, Elats)))
Wlats = [-5, -5, 5, 5]
Wlons = [-150,-90,-90,-150]
Wring = LinearRing(list(zip(Wlons, Wlats)))

# Read in Files 

In [3]:
## Set Paths

In [4]:
pwd

'/glade/u/home/calipfleger/stats/PWC_Paper/pwc_clean'

In [5]:
basedir_atm = '/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/'
file_prefix= '.cam.h0.' 
i_model = '/b.ie12.B1850C5CN.f19_g16.LME.' ## read in 2&3 iLME Ensembles 
i_model1 = '/b.ie12.B1850CN.f19_g16.' ## read in iLME ens 1
model = '/b.e11.BLMTRC5CN.f19_g16.'

# Import Packages

In [6]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import scipy.stats as stats
import xarray as xr
from tqdm import tqdm
import os
from pprint import pprint
#import nctoolkit
# colors for lines (color blind friendly colors: https://gist.github.com/thriveth/8560036)
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

import matplotlib as mpl  # # Plotting
import matplotlib.pyplot as plt  # python plotting package
import matplotlib.colors as mcolors
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from pandas.plotting import register_matplotlib_converters
from colorspacious import cspace_converter
mpl.rcParams['figure.dpi'] = 300

from collections import namedtuple
from collections import OrderedDict
cmaps = OrderedDict()

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from colorspacious import cspace_converter

In [7]:
import xarray as xr
import numpy as np
import cftime
import pandas as pd
from numpy import *
from scipy import stats
from netCDF4 import Dataset as nc
from netCDF4 import num2date
import netCDF4
import datetime
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt #python plotting package
from matplotlib import cm
import matplotlib.colors as mcolors
from colorspacious import cspace_converter
from collections import OrderedDict
cmaps = OrderedDict()
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import nctoolkit
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cpf
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely import geometry
from collections import namedtuple
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon

# Calculations and Functions 

In [8]:
# CESM converting month 01 to January 
from itertools import product
from cftime import DatetimeNoLeap
year = np.linspace(0,1872, 1872) 
dates = [DatetimeNoLeap(year, month, 1) for year, month in product(range(1850, 2006), range(1, 13))]
da = xr.DataArray(np.arange(1872), coords=[dates], dims=['time'], name='time')

In [9]:
## Set control year 
i_cntlyear = np.linspace(0,2400, 2400) 
i_cntldates = [DatetimeNoLeap(i_cntlyear, month, 1) for i_cntlyear, month in product(range(650, 850), range(1, 13))]
i_cntlda =xr.DataArray(np.arange(2400), coords=[i_cntldates], dims=['time'], name='time')

In [10]:
# define grid cell weighting to acount for grid cells in extratropics 
def weights(w):
    coslat = np.cos(np.deg2rad(w.lat)) #  Take the cosine of latitude (first converting to radians)
    weight_factor = coslat / coslat.mean(dim='lat')#  And divide by its mean value
    computed_weight = w * weight_factor # .mean(dim=('time', 'lon', 'lat'))
    return computed_weight

In [11]:
## Calculate Ensemble average 
def ensav(e):
    ens_avg = e.mean(dim = 'ensemble')
    return ens_avg

In [12]:
# Code Dr. Samantha Stevenson UCSB
#mask_lon = (mytos.lon >= regbox[2]) & (mytos.lon <= regbox[3])
#mask_lat = (mytos.lat >= regbox[0]) & (mytos.lat <= regbox[1])
#hsst=mytos.where(mask_lon & mask_lat, drop=True).squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

In [13]:
# PWC Index
def compute_index(c):
    east = c.sel(lat=slice(-5,5), lon=slice(210,270))
    west = c.sel(lat=slice(-5,5), lon=slice(90,150))
    #east = c.sel(lat=slice(-10,10), lon=slice(190,250))
    #west = c.sel(lat=slice(-10,10), lon=slice(90,150))
    avg_east = east.mean(('lat', 'lon'))
    avg_west =  west.mean(('lat', 'lon'))
    clim_index = (avg_east- avg_west).compute()
    return clim_index

In [14]:
# Function for meridional mean and lat lon range 
def mer(m):
    #m_we = weights(m)
    mer_lat= m.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270))
    mer_clim = clim(mer_lon)
    mer_time = mer_clim.mean(dim='time')
    mer_avg = mer_time#.mean(dim = 'ensemble')
    return mer_avg

In [15]:
## Calculate Anomaly relative to Climatology for variable 
def clim(tc): 
    #units = tc/100 #convert d18O 
    #convert precip 1000*24*3600
    c = tc.assign_coords({'time':(da.time)}) #convert time dimestion to have 01 be january 
    clim_var = c.groupby('time.month').mean('time') #calculate monthly climatology
    clim_anom = c.groupby('time.month') - clim_var # calculate the anomaly relative to the climatology 
    #anom_weights = weights(clim_anom) #weight functino to account for lat grid size 
    return  clim_anom #return the anomaly calcuated by the anomaly with time and lat conversion 

In [16]:
## Calculate Anomaly relative to Climatology for variable 
def clim_cntl(tc): 
    #units = tc/100 #convert d18O 
    #convert precip 1000*24*3600
    c = tc.assign_coords({'time':(i_cntlda.time)}) #convert time dimestion to have 01 be january 
    clim_var = c.groupby('time.month').mean('time') #calculate monthly climatology
    clim_anom = c.groupby('time.month') - clim_var # calculate the anomaly relative to the climatology 
    #anom_weights = weights(clim_anom) #weight functino to account for lat grid size 
    return clim_anom #return the anomaly calcuated by the anomaly with time and lat conversion 

In [17]:
## Calculate Ensemble average 
def clim_avg(ca):
    #convert precip 1000*24*3600
    c = ca.assign_coords({'time':(da.time)})
    time_avg = c.mean('time')
    ens_avg = time_avg.mean(dim = 'ensemble')
    return ens_avg

In [18]:
# Calculate Correlation Coefficent using xr.corr 
def cor(g,t):
    ds= []
    fcor =xr.corr(g,t, dim = 'time') # g is the global data set and t is the time series index 
    ds.append(fcor) # add correlation coeefiecent to dataset 
    cords= xr.concat(ds, dim='ensemble')  # create ensmenble dimesion for correlation coeeficent 
    cor_avg = cords.mean('ensemble') # calculate ensemble average for correlation coeeficent 
    return cor_avg 

In [19]:
def l_trend(var,lon,lat,time,sig):
    nlon=len(lon)
    nlat=len(lat)
    nt=len(time)
    vart=zeros(nlat*nlon)
    varp=zeros(nlat*nlon)
    if len(var.shape)== 3:        
        var1=reshape(var,(nt,nlat*nlon)) 
        for i in range(nlat*nlon):
            v=var1[:,i]  
            vart[i], intercept, r_value, varp[i], std_err=stats.linregress(time,v) # slope, intercept, r, p, std_error
        vart=reshape(vart,(nlat,nlon))
        varp=reshape(varp,(nlat,nlon))  
        vt.append(vart)
        vp.append(varp)
        slope = np.stack(vt, axis=0)#, dim='ensemble')
        pvalue = np.stack(vp, axis=0)#, dim='ensemble')
        sig_pval = ma.masked_greater(pvalue, sig)
        pval_avg = avg(sig_pval)
        slope_avg = avg(slope)
        return (slope, sig_pval, slope_avg, pval_avg) 

In [20]:
print('Index Files: ' +  model + period ,''+ region )

Index Files: /b.e11.BLMTRC5CN.f19_g16.Post Industrial PWC Index 


# Read in Index Files: PSL

In [21]:
var = 'U' #set output variable name for reading in file 

In [30]:
%%time
 #set output variable name for reading in file %%time
ens = '' #Full forcing ensemble 
file_suffix= '.185001-200512.nc'
ds = []# initialise array:
for member in tqdm(range(1,2)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
    
for member in tqdm(range(2,3+1)): #loop to read in the 1-9 ensemble members 
    id = str(member)
    file = id+file_prefix+var+ file_suffix # set file path 
    member = xr.open_dataset(basedir_atm + var+ i_model +ens +'00' +file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)

for member in tqdm(range(1,9+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
    
for member in tqdm(range(10,13+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ model+ens +'0'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
    
var_fullU = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)})#create ensemble dimension after reading in all the ensemble members

100%|██████████| 4/4 [00:14<00:00,  3.52s/it]


CPU times: user 2min 45s, sys: 6.39 s, total: 2min 51s
Wall time: 3min 33s


In [39]:
%%time
ens = 'GHG.'  # GHG ensemble 
file_suffix= '.185001-200512.nc'
ds = [] # initialise array:
for member in tqdm(range(1,2)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
for member in tqdm(range(3,4+1)): #add in 4th ensmble when permission is not denied 
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ model +ens +'00' +file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
var_ghgU = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)}) 

  0%|          | 0/1 [00:06<?, ?it/s]


AttributeError: 'Dataset' object has no attribute 'U'

In [33]:
%%time
ens = 'O3AER.'
file_suffix= '.185001-200512.nc'
ds = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)

ens = 'OZONE_AER.'
for member in tqdm(range(1,5+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)

var_ozaU = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)})  

100%|██████████| 5/5 [00:12<00:00,  2.50s/it]


CPU times: user 1min 45s, sys: 3.61 s, total: 1min 48s
Wall time: 2min 6s


In [38]:
%%time
ens = 'LULC.'  #Lulc ensemble 
file_suffix= '.169001-200512.nc'
ds = []
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(time = slice('1850-02-01', '2006-01-01')).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
ens = 'LULC_HurttPongratz.'
file_suffix= '.185001-200512.nc'
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ model+ ens + '00'+file).sel(time = slice('1850-02-01', '2006-01-01')).sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)
    ds.append(member)
var_lulcU = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)}) 

  0%|          | 0/3 [00:00<?, ?it/s]


AttributeError: 'Dataset' object has no attribute 'U'

In [34]:
%%time
var = 'V' #set output variable name for reading in file %%time
ens = '' #Full forcing ensemble 
file_suffix= '.185001-200512.nc'
ds = []# initialise array:
for member in tqdm(range(1,2)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
    
for member in tqdm(range(2,3+1)): #loop to read in the 1-9 ensemble members 
    id = str(member)
    file = id+file_prefix+var+ file_suffix # set file path 
    member = xr.open_dataset(basedir_atm + var+ i_model +ens +'00' +file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)

for member in tqdm(range(1,9+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
    
for member in tqdm(range(10,13+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ model+ens +'0'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
    
var_full= xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)}) 

100%|██████████| 4/4 [00:23<00:00,  5.76s/it]


CPU times: user 3min 7s, sys: 7.12 s, total: 3min 15s
Wall time: 4min 27s


In [35]:
%%time
ens = 'GHG.'  # GHG ensemble 
file_suffix= '.185001-200512.nc'
ds = [] # initialise array:
for member in tqdm(range(1,2)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
for member in tqdm(range(3,4+1)): #add in 4th ensmble when permission is not denied 
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ model +ens +'00' +file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
var_ghg = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)}) 

100%|██████████| 3/3 [00:11<00:00,  3.94s/it]


CPU times: user 24.8 s, sys: 904 ms, total: 25.8 s
Wall time: 43.7 s


In [36]:
%%time
ens = 'O3AER.'
file_suffix= '.185001-200512.nc'
ds = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)

ens = 'OZONE_AER.'
for member in tqdm(range(1,5+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)

var_oza= xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)}) 

100%|██████████| 5/5 [00:21<00:00,  4.36s/it]


CPU times: user 1min 45s, sys: 4.08 s, total: 1min 49s
Wall time: 2min 16s


In [40]:
%%time
ens = 'LULC.'  #Lulc ensemble 
file_suffix= '.169001-200512.nc'
ds = []
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(time = slice('1850-02-01', '2006-01-01')).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
ens = 'LULC_HurttPongratz.'
file_suffix= '.185001-200512.nc'
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ model+ ens + '00'+file).sel(time = slice('1850-02-01', '2006-01-01')).sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)
    ds.append(member)
var_lulc = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)}) 

100%|██████████| 3/3 [00:09<00:00,  3.12s/it]


CPU times: user 1min 11s, sys: 2.41 s, total: 1min 14s
Wall time: 1min 28s


In [42]:
icntl_indexU = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/U/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.U.065001-084912.nc').sel(lat=slice(-35,35),lon=slice(75,290)).U.squeeze().isel(lev=23)

In [44]:
icntl_indexV = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/V/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.V.065001-084912.nc').sel(lat=slice(-35,35),lon=slice(75,290)).V.squeeze().isel(lev=23)

In [ ]:
variable = 'V'
var_full.to_netcdf(variable+'full.nc')
var_ghg.to_netcdf(variable+'ghg.nc')
var_oza.to_netcdf(variable+'oza.nc')
var_lulc.to_netcdf(variable+'lulc.nc')
icntl_indexV.to_netcdf(variable+'cntlV.nc')

In [ ]:
variable = 'U'
var_fullU.to_netcdf(variable+'full.nc')
var_ghgU.to_netcdf(variable+'ghg.nc')
var_ozaU.to_netcdf(variable+'oza.nc')
var_lulcU.to_netcdf(variable+'lulc.nc')
icntl_indexU.to_netcdf(variable+'cntlU.nc')

# Read in Second Variable: dD Files 

## Calculate dD for full forcing

In [ ]:
%%time
ens = ''
file_suffix= '.185001-200512.nc'

var = 'H218Or'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218Or = xr.concat(dsf, dim='ensemble')

var = 'H218OL'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218OL = xr.concat(dsf, dim='ensemble')

var = 'H218OS'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218OS = xr.concat(dsf, dim='ensemble')

var = 'H218OR'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218OR = xr.concat(dsf, dim='ensemble')

var = 'H218Os'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218Os = xr.concat(dsf, dim='ensemble')

var = 'H218OV'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218OV = xr.concat(dsf, dim='ensemble')

var = 'H218OI'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H218OI = xr.concat(dsf, dim='ensemble')

In [ ]:
post_H218O = post_H218Or + post_H218OR +post_H218Os +post_H218OS + post_H218OI + post_H218OL +post_H218OV # adll all 18O componen

In [ ]:
del post_H218Or 
del post_H218OR 
del post_H218Os 
del post_H218OS 
del post_H218OI 
del post_H218OL
del post_H218OV

In [ ]:
%%time
ens = ''
file_suffix= '.185001-200512.nc'

var = 'H216Or'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216Or = xr.concat(dsf, dim='ensemble')

var = 'H216OL'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216OL = xr.concat(dsf, dim='ensemble')

var = 'H216OS'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216OS = xr.concat(dsf, dim='ensemble')

var = 'H216OR'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216OR = xr.concat(dsf, dim='ensemble')

var = 'H216Os'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216Os = xr.concat(dsf, dim='ensemble')

var = 'H216OV'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216OV = xr.concat(dsf, dim='ensemble')

var = 'H216OI'
dsf = []# initialise array:
for member in range(1,2):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)   
for member in range(2,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsf.append(member)     
post_H216OI = xr.concat(dsf, dim='ensemble')

In [ ]:
post_H216O= post_H216Or + post_H216OR +post_H216Os +post_H216OS + post_H216OI + post_H216OL + post_H216OV # add all 16O componets 

In [ ]:
del post_H216Or
del post_H216OR 
del post_H216Os 
del post_H216OS 
del post_H216OI 
del post_H216OL 
del post_H216OV 

In [ ]:
var_full =  (((post_H218O/post_H216O)-1)*1000)

## Read in GHG files and Calculate dD

In [ ]:
%%time 
## GHG
ens = 'GHG.'  #set ensemble 

var = 'H218OL'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218OL_ghg = xr.concat(dsg, dim='ensemble')

var = 'H218Or'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218Or_ghg = xr.concat(dsg, dim='ensemble')

var = 'H218OS'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218OS_ghg = xr.concat(dsg, dim='ensemble')

var = 'H218OR'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218OR_ghg = xr.concat(dsg, dim='ensemble')

var = 'H218Os'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218Os_ghg = xr.concat(dsg, dim='ensemble')

var = 'H218OV'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218OV_ghg = xr.concat(dsg, dim='ensemble')

var = 'H218OI'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H218OI_ghg = xr.concat(dsg, dim='ensemble')

In [ ]:
%%time 
post_H218O_ghg = post_H218Or_ghg + post_H218OR_ghg +post_H218Os_ghg +post_H218OS_ghg + post_H218OI_ghg + post_H218OL_ghg +post_H218OV_ghg # adll all 18O componen

In [ ]:
del post_H218Or_ghg 
del post_H218OR_ghg 
del post_H218Os_ghg  
del post_H218OS_ghg  
del post_H218OI_ghg  
del post_H218OL_ghg 
del post_H218OV_ghg 

In [ ]:
%%time
## GHG
var = 'H216OL'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216OL_ghg = xr.concat(dsg, dim='ensemble')

var = 'H216Or'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216Or_ghg = xr.concat(dsg, dim='ensemble')

var = 'H216OS'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216OS_ghg = xr.concat(dsg, dim='ensemble')

var = 'H216OR'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216OR_ghg = xr.concat(dsg, dim='ensemble')

var = 'H216Os'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216Os_ghg = xr.concat(dsg, dim='ensemble')

var = 'H216OV'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216OV_ghg = xr.concat(dsg, dim='ensemble')

var = 'H216OI'
dsg = []
for member in (1,3,4):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model +ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsg.append(member)
post_H216OI_ghg = xr.concat(dsg, dim='ensemble')

In [ ]:
%%time
post_H216O_ghg = post_H216Or_ghg + post_H216OR_ghg +post_H216Os_ghg +post_H216OS_ghg + post_H216OI_ghg + post_H216OL_ghg +post_H216OV_ghg # add all 16O componets 

In [ ]:
del post_H216Or_ghg # rC is convective rain 
del post_H216OR_ghg # L - large scale 
del post_H216Os_ghg #SC = snow rates convective?
del post_H216OS_ghg # SL = snow rates large scale?
del post_H216OI_ghg #ice 
del post_H216OL_ghg #liquid 
del post_H216OV_ghg #vapor 

In [ ]:
%%time
var_ghg =  (((post_H218O_ghg/post_H216O_ghg)-1)*1000)

## Read in OZA files and Calculate dD

In [ ]:
%%time
# OZA
ens = 'O3AER.'
var = 'H218Or'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218Or_oza = xr.concat(dso, dim='ensemble')

var = 'H218OL'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218OL_oza = xr.concat(dso, dim='ensemble')

var = 'H218OS'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218OS_oza = xr.concat(dso, dim='ensemble')

var = 'H218OR'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218OR_oza = xr.concat(dso, dim='ensemble')

var = 'H218Os'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218Os_oza = xr.concat(dso, dim='ensemble')

var = 'H218OV'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218OV_oza = xr.concat(dso, dim='ensemble')

var = 'H218OI'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H218OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H218OI_oza = xr.concat(dso, dim='ensemble')

In [ ]:
%%time
post_H218O_oza = post_H218Or_oza + post_H218OR_oza +post_H218Os_oza +post_H218OS_oza+ post_H218OI_oza + post_H218OL_oza+post_H218OV_oza # adll all 18O componen

In [ ]:
del post_H218Or_oza 
del post_H218OR_oza 
del post_H218Os_oza  
del post_H218OS_oza  
del post_H218OI_oza  
del post_H218OL_oza 
del post_H218OV_oza 

In [ ]:
%%time
# OZA
var = 'H216Or'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216Or_oza = xr.concat(dso, dim='ensemble')

var = 'H216OL'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216OL_oza = xr.concat(dso, dim='ensemble')

var = 'H216OS'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216OS_oza = xr.concat(dso, dim='ensemble')

var = 'H216OR'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216OR_oza = xr.concat(dso, dim='ensemble')

var = 'H216Os'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216Os_oza = xr.concat(dso, dim='ensemble')

var = 'H216OV'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216OV_oza = xr.concat(dso, dim='ensemble')

var = 'H216OI'
dso = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290)).H216OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dso.append(member)   
post_H216OI_oza = xr.concat(dso, dim='ensemble')

In [ ]:
%%time
H216O_oza = post_H216Or_oza + post_H216OR_oza +post_H216Os_oza +post_H216OS_oza +post_H216OI_oza + post_H216OL_oza +post_H216OV_oza # add all 16O componets 

In [ ]:
del post_H216Or_oza # rC is convective rain 
del post_H216OR_oza # L - large scale 
del post_H216Os_oza #SC = snow rates convective?
del post_H216OS_oza # SL = snow rates large scale?
del post_H216OI_oza #ice 
del post_H216OL_oza #liquid 
del post_H216OV_oza #vapor 

In [ ]:
%%time
var_oza =  (((post_H218O_oza/H216O_oza)-1)*1000)

## Read in LULC files and Calculate dD

In [ ]:
%%time
ens = 'LULC.'  #Lulc ensemble  
file_suffix= '.169001-200512.nc'

var = 'H218OI'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218OI_lulc = xr.concat(dsl, dim='ensemble')


var = 'H218Or'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218Or_lulc = xr.concat(dsl, dim='ensemble')

var = 'H218OS'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218OS_lulc = xr.concat(dsl, dim='ensemble')

var = 'H218OR' 
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218OR_lulc = xr.concat(dsl, dim='ensemble')


var = 'H218Os'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218Os_lulc = xr.concat(dsl, dim='ensemble')

var = 'H218OV'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218OV_lulc = xr.concat(dsl, dim='ensemble')

var = 'H218OL'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H218OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H218OL_lulc = xr.concat(dsl, dim='ensemble')

In [ ]:
%%time 
post_H218O_lulc = post_H218Or_lulc + post_H218OR_lulc +post_H218Os_lulc +post_H218OS_lulc+ post_H218OI_lulc + post_H218OL_lulc+post_H218OV_lulc # adll all 18O componen

In [ ]:
del post_H218Or_lulc 
del post_H218OR_lulc 
del post_H218Os_lulc  
del post_H218OS_lulc  
del post_H218OI_lulc  
del post_H218OL_lulc 
del post_H218OV_lulc 

In [ ]:
var = 'H216OI'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216OI_lulc = xr.concat(dsl, dim='ensemble')


var = 'H216Or'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216Or_lulc = xr.concat(dsl, dim='ensemble')

var = 'H216OS'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216OS_lulc = xr.concat(dsl, dim='ensemble')

var = 'H216OR' 
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216OR_lulc = xr.concat(dsl, dim='ensemble')


var = 'H216Os'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216Os_lulc = xr.concat(dsl, dim='ensemble')

var = 'H216OV'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216OV_lulc = xr.concat(dsl, dim='ensemble')

var = 'H216OL'
dsl = []
for member in range(1,3+1):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-30,30),lon=slice(75,290), time = slice('1850-02-01', '2006-01-01')).H216OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)
    dsl.append(member)
post_H216OL_lulc = xr.concat(dsl, dim='ensemble')

In [ ]:
%%time 
post_H216O_lulc = post_H216Or_lulc + post_H216OR_lulc +post_H216Os_lulc +post_H216OS_lulc+ post_H216OI_lulc + post_H216OL_lulc+post_H216OV_lulc # adll all 16O componen

In [ ]:
del post_H216Or_lulc 
del post_H216OR_lulc 
del post_H216Os_lulc  
del post_H216OS_lulc  
del post_H216OI_lulc  
del post_H216OL_lulc 
del post_H216OV_lulc 

In [ ]:
%%time
var_lulc =  (((post_H218O_lulc/post_H216O_lulc)-1)*1000)

## Read in Control files and Calculate dD

In [ ]:
## Control 
##cntl_PREC_H218O = cntl_PRECRC_H218Or + cntl_PRECRL_H218OR +cntl_PRECSC_H218Os +cntl_PRECSL_H218OS
#cntl_PREC_H216O = cntl_PRECRC_H216Or + cntl_PRECRL_H216OR +cntl_PRECSC_H216Os +cntl_PRECSL_H216OS
#cntl_var = (((cntl_PREC_H218O/cntl_PREC_H216O)-1)*1000)

In [ ]:
## dH Control 
cntl_H218Or = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218Or/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218Or.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216Or = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216Or/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216Or.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216Or.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

cntl_H218OR = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OR/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OR.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OR = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OR/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OR.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OR.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

cntl_H218Os = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218Os/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218Os.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216Os = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216Os/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216Os.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216Os.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

cntl_H218OS = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OS/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OS.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OS = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OS/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OS.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OS.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

cntl_H218OI = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OI/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OI.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OI = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OI/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OI.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OI.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

cntl_H218OL = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OL/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OL.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OL = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OL/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OL.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OL.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

cntl_H218OV = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H218OV/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H218OV.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H218OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23) #AttributeError: 'CFTimeIndex' object has no attribute '_cache'
cntl_H216OV = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/H216OV/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.H216OV.065001-084912.nc').sel(lat=slice(-30,30),lon=slice(75,290)).H216OV.squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

In [ ]:
## Control 
cntl_H218O = cntl_H218Or + cntl_H218OR +cntl_H218Os +cntl_H218OS +cntl_H218OI +cntl_H218OL +cntl_H218OV
cntl_H216O = cntl_H216Or + cntl_H216OR +cntl_H216Os +cntl_H216OS +cntl_H216OI +cntl_H216OL +cntl_H216OV
var_cntl = (((cntl_H218O/cntl_H216O)-1)*1000)

In [ ]:
del cntl_H218Or 
del cntl_H216Or 
del cntl_H218OR 
del cntl_H216OR 
del cntl_H218Os 
del cntl_H216Os 
del cntl_H218OS
del cntl_H216OS 
del cntl_H218OI 
del cntl_H216OI 
del cntl_H218OL
del cntl_H216OL 
del cntl_H218OV 
del cntl_H216OV 

In [ ]:
def slp_index(r):
    r_lat= r.sel(lat=slice(-5, 5)).mean(dim='lat')
    r_mer =  r_lat.sel(lon=slice(110, 270))
    return r_mer

In [ ]:
def reg_mer(r):
    r_lat= r.sel(lat=slice(-5, 5)).mean(dim='lat')
    r_mer =  r_lat.sel(lon=slice(110, 270))
    r_mer18 = r_mer.isel(lev = 18)
    return r_mer, r_mer18

In [ ]:
## Calculate Anomaly relative to Climatology for variable 
def clim_anom(v, p): 
    units = v/100 #convert to hPa d18O #convert precip 1000*24*3600
    units_p = p/100 #convert to hPa d18O #convert precip 1000*24*3600
   
    pi = units_p.assign_coords({'time':(i_cntlda.time)}) #convert time dimestion to have 01 be january 
    vv = units.assign_coords({'time':(da.time)}) #convert time dimestion to have 01 be january 

    del units_p
    del units

    # Group by month dimension
    pi_month =pi.groupby('time.month').mean('time') #calculate monthly climatology
    clim_month = vv.groupby('time.month').mean('time') #calculate monthly climatology #return

    # Calcualte anomaly relative to climatology 
    pi_anom = pi.groupby('time.month') - pi_month # 1 calculate the anomaly relative to the climatology x4
    clim_anom = vv.groupby('time.month') - clim_month #2 calculate the anomaly relative to the climatology x4
    
    del pi_month
    del clim_month 

    # Ensemble average effect of forcing 
    eforcing_avg = (vv - pi).mean(dim = 'ensemble') # 3netcdf x4
    # Meridtrional Average 
    clim_mer, clim_mer18 = reg_mer(vv) # x4
    pi_mer, pi_mer18 = reg_mer(pi) #x1

    #del vv
    #del pi 
    # Ensemble averagefor climatigy realtive to anomaly and effect of forcing anomalies 
    anom_eforcing_avg = (clim_anom - pi_anom).mean(dim = 'ensemble') #4 return  #netcdfx4
    clim_anomavg = clim_anom.mean(dim = 'ensemble')  #5 return x4
    
    # Vertical Slice 
    clim_anom_mer, clim_anom_mer18 = reg_mer(clim_anom) #6, 7 x4
    pi_anom_mer, pi_anom_mer18 = reg_mer(pi_anom) #8, 9 x1
    
    # all levels  tropical domain  effect of forcing climatology and climatology anomaly 
    eforcing_mer = (clim_mer - pi_mer).mean(dim = 'ensemble') # 10 return x4
    anom_eforcing_mer = (clim_anom_mer - pi_anom_mer).mean(dim = 'ensemble') # 11 return x4

    del clim_mer
    del pi_mer

    # level 18 effect of forcing climatology and climatology anomaly 
    
    eforcing_mer18 = (clim_mer18 - pi_mer18).mean(dim = 'ensemble') #12 return x4
    anom_eforcing_mer18 = (clim_anom_mer18 - pi_anom_mer18).mean(dim = 'ensemble') # 13 return x4

  
    return vv, pi, clim_anom, pi_anom, eforcing_avg, anom_eforcing_avg,clim_anomavg, anom_eforcing_mer, eforcing_mer, anom_eforcing_mer,anom_eforcing_mer18 

In [ ]:
# Function for meridional mean and lat lon range 
def mer_series(m):
    #m_we = weights(m)
    c = m.assign_coords({'time':(da.time)}) #convert time dimestion to have 01 be january 
    mer_lat= c.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270)).mean(dim='lon')
    #mer_clim = clim(mer_lon)
    mer_time = mer_lon.mean(dim='time')
    mer_avg = mer_time.mean(dim = 'ensemble')
    mer_series = mer_avg.to_series()
    return mer_series

# Select Atmospheric Pressure Level (524 hpa)

In [ ]:
var_ghg18 = var_ghg.isel(lev = 18)
var_full18 = var_full.isel(lev = 18)
var_oza18 = var_oza.isel(lev = 18)
var_lulc18 = var_full.isel(lev = 18)
var_cntl18 = var_cntl.isel(lev = 18)

# Calculate Climatology 
- Convert CESM data to start at 1 January, not python index
- Weight by grid cell 

In [ ]:
%%time
# Calculate anomaly relative to climatology for global variable forcing files 
anom_var18 = clim(var_full18)#var_full.isel(lev = 18))
anom_var_ghg18 = clim(var_ghg18)
anom_var_lulc = clim(var_lulc)
anom_var_oza18 = clim(var_oza18)
anom_var_cntl18 = clim_cntl(var_cntl18)

In [ ]:
%%time
# Calculate anomaly relative to climatology for global variable forcing files 
anom_var = clim(var_full)#var_full.isel(lev = 18))
anom_var_ghg = clim(var_ghg)
#anom_var_lulc = clim(var_lulc)
anom_var_oza = clim(var_oza)
anom_var_cntl = clim_cntl(var_cntl)

In [ ]:
%%time
# Calculate anomaly relative to climatology for index forcing files 
anom_index = clim(post_index)
anom_index_ghg = clim(post_index_ghg)
#anom_index_lulc = clim(index_lulc)
anom_index_oza = clim(post_index_oza)
anom_index_cntl = clim_cntl(icntl_index)

In [ ]:
# Calculate anomaly relative to climatology for index forcing files 
#anom_index_avg = clim(post_index).mean(dim = 'time')
#anom_index_ghg_avg  = clim(post_index_ghg).mean(dim = 'time')
#anom_index_lulc = clim(index_lulc)
#anom_index_oza_avg  = clim(post_index_oza).mean(dim = 'time')
#anom_index_cntl_avg  = clim_cntl(icntl_index).mean(dim = 'time')

In [ ]:
def tens (tt):
    test = tt.mean(dim = 'time')
    test2 = test.mean(dim = 'ensemble')
    return test2

# Calculate ensemble average climatology anomaly s 

# Mapping the Climatological Average

In [ ]:
#anom_var_cntl_avg  = tens(anom_var_cntl)

In [ ]:
%%time
# Calculate anomaly relative to climatology for global variable forcing files 
anom_var_avg18 = tens(anom_var18)#var_full.isel(lev = 18))
anom_var_ghg_avg18  = tens(anom_var_ghg18)
#anom_var_lulc = clim(var_lulc)
anom_var_oza_avg18 = tens(anom_var_oza18)
#anom_var_cntl_avg18  = tens(anom_var_cntl18)

In [ ]:
clim_maps = [anom_var_avg18,anom_var_ghg_avg18, anom_var_oza_avg18 ]

In [ ]:
%%time
# Calculate anomaly relative to climatology for global variable forcing files 
anom_var_avg = tens(anom_var)#var_full.isel(lev = 18))
anom_var_ghg_avg  = tens(anom_var_ghg)
#anom_var_lulc = clim(var_lulc)
anom_var_oza_avg  = tens(anom_var_oza)
#anom_var_cntl_avg  = tens(anom_var_cntl)

# Vertical Slice: Meridional Mean and Vertical Profile

In [ ]:
def vertplot(v):
    # Plotting the average of the meridional mean of the vertical velocity and pressure contour lines
    ax = fig.add_subplot(2,2,i+1)
    contour = v[i].plot.contourf(ax=ax, levels=30, cmap='RdBu_r', vmin= -0.000002, vmax= 0.000002)
    pressure_contour = v[i].plot.contour(ax=ax, levels=10, colors='k', linewidths=0.5)
    # Add contour line labels with numeric values
    ax.clabel(pressure_contour, fmt='%1.2f', colors='k', fontsize=8)
    plt.xlabel('Longitude')
    plt.ylabel('Pressure (hPa)')
    plt.title(var_name[i] + ' Average Meridional Mean dH')
    #cbar = plt.colorbar(contour, label='Pa/s')  # Create the colorbar using the contour object
    #plt.gca().invert_yaxis()  # Flip the y-axis
    plt.show()

In [ ]:
def trop_plot(p):
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,p[i],cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,p[i], cmap='bwr', vmin=-1, vmax=1, transform=ccrs.PlateCarree())  
    #calculate significance of trends
    sig_p=xr.DataArray(data=p[i].values*np.sqrt((df-2)/(1-np.square(p[i].values))), 
                       dims=["lat","lon'"], coords=[lat, lon])
    sig_p.plot.contourf(ax=ax,levels = [-1*t95, -1*t90, t90, t95], colors='none', 
                        hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    ax.set_extent([75, 290, -30, 30], crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-20, -10, 0, 10, 20])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

In [ ]:
# Function for meridional mean and lat lon range 
def mer_clim(m):
    #m_we = weights(m)
    mer_lat= m.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270))
    mer_clim = clim(mer_lon)
    mer_time = mer_clim.mean(dim='time')
    mer_series= mer_time.mean(dim = 'ensemble')
    return mer_series

In [ ]:
# Function for meridional mean and lat lon range 
def mer_clim_cntl(m):
    c = m.assign_coords({'time':(i_cntlda.time)}) #convert time dimestion to have 01 be january 
    mer_lat= c.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270))
    mer_time = mer_lon.mean(dim='time')
    return mer_time

# 524 hPa 20-20 avg over 1850-2005

In [ ]:
%%time
series = mer_clim(var_full)#var_full.isel(lev = 18))
series_ghg  = mer_clim(var_ghg)
#anom_var_lulc = clim(var_lulc)
series_oza = mer_clim(var_oza)
anom_var_cntl_avgmer  = mer_clim_cntl(anom_var_cntl)

In [ ]:
clim_avg = [series, series_ghg, series_oza, anom_var_cntl_avgmer]

In [ ]:
def trop_plot(p):
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,p[i],cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,p[i], cmap='bwr', vmin=-1, vmax=1, transform=ccrs.PlateCarree())  
    #calculate significance of trends
    sig_p=xr.DataArray(data=p[i].values*np.sqrt((df-2)/(1-np.square(p[i].values))), 
                       dims=["lat","lon'"], coords=[lat, lon])
    sig_p.plot.contourf(ax=ax,levels = [-1*t95, -1*t90, t90, t95], colors='none', 
                        hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    ax.set_extent([75, 290, -30, 30], crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-20, -10, 0, 10, 20])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(clim_avg)):
    vertplot(clim_avg)
plt.suptitle('dD Correlation with ' + region, y =.65)
#plt.rcParams['font.size'] = '24'
plt.subplots_adjust(hspace=-1, wspace=0)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

In [ ]:
# Function for meridional mean and lat lon range 
def mer_series(m):
    #m_we = weights(m)
    c = m.assign_coords({'time':(da.time)}) #convert time dimestion to have 01 be january 
    mer_lat= c.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270)).mean(dim='lon')
    #mer_clim = clim(mer_lon)
    mer_time = mer_lon.mean(dim='time')
    mer_avg = mer_time.mean(dim = 'ensemble')
    mer_series = mer_avg.to_series()
    return mer_series

In [ ]:
# Function for meridional mean and lat lon range 
def mer_series_cntl(m):
    c = m.assign_coords({'time':(i_cntlda.time)}) #convert time dimestion to have 01 be january 
    mer_lat= c.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270)).mean(dim='lon')
    mer_time = mer_lon.mean(dim='time')
    mer_series = mer_time.to_series()
    return mer_series

In [ ]:
%%time
# Calculate anomaly relative to climatology for global variable forcing files 
series = mer_series(anom_var)#var_full.isel(lev = 18))
series_ghg  = mer_series(anom_var_ghg)
#anom_var_lulc = clim(var_lulc)
series_oza = mer_series(anom_var_oza)
series_cntl = mer_cntl(anom_var_cntl)

df = pd.DataFrame({'Full': series, 'GHG': series_ghg, 'OZA': series_oza})
df.plot(kind='bar', title ='dDV Anomalies as a function of height in the atmosphere 20N-20S').invert_yaxis()

In [ ]:
%%time
series_cntl  = mer_cntl(var_cntl)
series = mer_series(var_full)#var_full.isel(lev = 18))
series_ghg  = mer_series(var_ghg)
#anom_var_lulc = clim(var_lulc)
series_oza = mer_series(var_oza)

df = pd.DataFrame({'Full': series, 'GHG': series_ghg, 'OZA': series_oza})
df.plot(kind='bar', title ='dD as a function of height in the atmosphere 20N-20S').invert_yaxis()

In [ ]:
%%time

series =series - series_cntl
series_ghg =series_ghg - series_cntl
series_oza =series_oza - series_cntl

df = pd.DataFrame({'Full': series, 'GHG': series_ghg, 'OZA': series_oza})
df.plot(kind='bar', title ='dD as a function of height in the atmosphere 20N-20S \n (Forcing - PI CNTL)').invert_yaxis()

# Calculate PWC index 

In [ ]:
%%time
index = compute_index(anom_index)
index_ghg = compute_index(anom_index_ghg)
#index_lulc = compute_index(anom_index_lulc)
index_oza = compute_index(anom_index_oza)
index_cntl = compute_index(anom_index_cntl)

# Calculate Correlation with PWC Index

In [ ]:
cor_f = cor(anom_var, index)
cor_g = cor(anom_var_ghg, index_ghg)
#cor_l = cor(anom_var_lulc, pwc_lulc)
cor_o = cor(anom_var_oza, index_oza)
cor_var_cntl = cor(anom_var_cntl, index_cntl)
cors = [cor_f, cor_g, cor_o, cor_var_cntl]#make list of correlations to loop over 

In [ ]:
#SLP
#cor_fin = cor(anom_index, index)
#cor_gin= cor(anom_index_ghg, index_ghg)
#cor_lin = cor(anom_index_lulc, index_lulc)
#cor_oin = cor(anom_index_oza, index_oza)
#cor_cin = cor(anom_index_cntl, index_cntl)
#icor_PSL = cor(ianom_var_cntl, ipwc_cntl)
#corsin= [cor_fin, cor_gin,  cor_oin, cor_cin] #cor_lP,

In [ ]:
controls = [cor_var_cntl]

# Plotting Parameters

In [ ]:
lat = post_index.lat
lon = post_index.lon

In [ ]:
lent = len(cors)

In [ ]:
var_name = ['Full Forcing', 'GHG','Ozone-Aerosol', 'PI Cntl']#, 'ANTHRO']'LULC'

In [ ]:
df = 35 #degrees of freedom 
t90=stats.t.ppf(1-0.05, df-2) # testing for p-value 90% significant 
t95=stats.t.ppf(1-0.025, df-2) # testing for p-value 95% significant 

In [ ]:
def vertplot(v):
    # Plotting the average of the meridional mean of the vertical velocity and pressure contour lines
    fig, ax = plt.subplots(figsize=(20, 6))
    contour = v.plot.contourf(ax=ax, levels=30, cmap='RdBu_r', vmin= -0.000002, vmax= 0.000002)
    pressure_contour = v.plot.contour(ax=ax, levels=10, colors='k', linewidths=0.5)
    # Add contour line labels with numeric values
    ax.clabel(pressure_contour, fmt='%1.2f', colors='k', fontsize=8)

    plt.xlabel('Longitude')
    plt.ylabel('Pressure (hPa)')
    plt.title(var_name + ' Average Meridional Mean dH')
    #cbar = plt.colorbar(contour, label='Pa/s')  # Create the colorbar using the contour object
    #plt.gca().invert_yaxis()  # Flip the y-axis
    plt.show()

In [ ]:
def trop_plot(p):
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,p[i],cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,p[i], cmap='bwr', vmin=-1, vmax=1, transform=ccrs.PlateCarree())  
    #calculate significance of trends
    sig_p=xr.DataArray(data=p[i].values*np.sqrt((df-2)/(1-np.square(p[i].values))), 
                       dims=["lat","lon'"], coords=[lat, lon])
    sig_p.plot.contourf(ax=ax,levels = [-1*t95, -1*t90, t90, t95], colors='none', 
                        hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    ax.set_extent([75, 290, -30, 30], crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-20, -10, 0, 10, 20])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

In [ ]:
#cmap='BrBG' Brown to teal for precip dry/wet 
def gplot(p):
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,p[i],cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,p[i], cmap='bwr', vmin=-1, vmax=1, transform=ccrs.PlateCarree())  
    #calculate significance of trends
    sig_p=xr.DataArray(data=p[i].values*np.sqrt((df-2)/(1-np.square(p[i].values))), 
                       dims=["lat","lon'"], coords=[lat, lon])
    sig_p.plot.contourf(ax=ax,levels = [-1*t95, -1*t90, t90, t95], colors='none', 
                        hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

# Maps

In [ ]:
#List of what to plot
controls_in = [cor_in, cor_in, cor_in]

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(cors)):
    trop_plot(cors)
plt.suptitle('dD Correlation with ' + region, y =.65)
#plt.rcParams['font.size'] = '24'
plt.subplots_adjust(hspace=-1, wspace=0)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(cors)):
    gplot(cors)
plt.suptitle('dD Correlation with ' + region, y =.75)
#plt.rcParams['font.size'] = '24'
plt.subplots_adjust(hspace=0, wspace=0)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

# Correlation with PWC - PI variable correlation with PI CNTL 

In [ ]:
## Standard Map 
def globcntl_plot(p, pp):
    #p = p[i] - cor_PSL[i] # subtract control run correlation 
    w = p[i] - pp
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,w,cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,w, cmap='bwr', vmin=-0.2, vmax=0.2, transform=ccrs.PlateCarree())  
    #calculate significance of trends
    sig_p=xr.DataArray(data=p[i].values*np.sqrt((df-2)/(1-np.square(p[i].values))), 
                       dims=["lat","lon'"], coords=[lat, lon])
    sig_p.plot.contourf(ax=ax,levels = [-1*t95, -1*t90, t90, t95], colors='none', 
                        hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

In [ ]:
## Standard Map 
def tropcntl_plot(p, pp):
    #p = p[i] - cor_PSL[i] # subtract control run correlation 
    w = p[i] - pp
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,w,cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,w, cmap='bwr', vmin=-0.2, vmax=0.2, transform=ccrs.PlateCarree())  
    #calculate significance of trends
    sig_p=xr.DataArray(data=p[i].values*np.sqrt((df-2)/(1-np.square(p[i].values))), 
                       dims=["lat","lon'"], coords=[lat, lon])
    sig_p.plot.contourf(ax=ax,levels = [-1*t95, -1*t90, t90, t95], colors='none', 
                        hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    ax.set_extent([75, 290, -30, 30], crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-20, -10, 0, 10, 20])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(cors)):
    tropcntl_plot(cors, cor_var_cntl)
plt.suptitle('d18op' + ' Post Industrial PWC - PI CNTL Correlation', y =.95)
plt.subplots_adjust(hspace=0, wspace=0)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(cors)):
    globcntl_plot(cors, cor_var_cntl)
plt.suptitle('d18op' + ' Post Industrial PWC - PI CNTL Correlation', y =.95)
plt.subplots_adjust(hspace=0, wspace=0)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

## 95% significance trend of grid points 

In [ ]:
def avg(r):
    var_avg = r.mean(axis =0) 
    return(var_avg)

In [ ]:
def l_trend(var,lon,lat,time,sig):
    nlon=len(lon)
    nlat=len(lat)
    nt=len(time)
    vart=zeros(nlat*nlon)
    varp=zeros(nlat*nlon)
    if len(var.shape)== 3:        
        var1=reshape(var,(nt,nlat*nlon)) 
        for i in range(nlat*nlon):
            v=var1[:,i]  
            vart[i], intercept, r_value, varp[i], std_err=stats.linregress(time,v) # slope, intercept, r, p, std_error
        vart=reshape(vart,(nlat,nlon))
        varp=reshape(varp,(nlat,nlon))  
        vt.append(vart)
        vp.append(varp)
        slope = np.stack(vt, axis=0)#, dim='ensemble')
        pvalue = np.stack(vp, axis=0)#, dim='ensemble')
        sig_pval = ma.masked_greater(pvalue, sig)
        pval_avg = avg(sig_pval)
        slope_avg = avg(slope)
        return (slope, sig_pval, slope_avg, pval_avg) 

In [ ]:
%%time
vt = []
vp = []
for ww in range(len(var_full)):
    trend_full18, pval_full18, full_avg18, full_avgp18 = l_trend(anom_var18[ww,:,:,:].values,lon,lat,year,sig=0.05)

In [ ]:
%%time
vt = []
vp = []
for ww in range(len(var_ghg)):
    trend_ghg, pval_ghg, ghg_avg, ghg_avgp = l_trend(anom_var_ghg18[ww,:,:,:].values,lon,lat,year,sig=0.05)

In [ ]:
%%time
vt = []
vp = []
for ww in range(len(var_oza)):
    trend_oza, pval_oza, oza_avg, oza_avgp = l_trend(anom_var_oza18[ww,:,:,:].values,lon,lat,year,sig=0.05)

In [ ]:
lt = [full_avg18, ghg_avg, oza_avg]
lt_sig = [full_avgp18, ghg_avgp, oza_avgp]

In [ ]:
def trop_plot_trend(p,s):
    ax = fig.add_subplot(2,2,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,p[i],cmap='bwr', add_colorbar=False, transform=ccrs.PlateCarree())
    plt.pcolor(lon,lat,p[i], cmap='bwr', vmin=-1, vmax=1, transform=ccrs.PlateCarree())  
    plt.contourf(lon,lat,s[i], colors='none',  hatches=['..', None, None, None, '..'], extend='both', add_colorbar=False, 
                        transform=ccrs.PlateCarree())
    ax.set_extent([75, 290, -30, 30], crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.xlines = False
    gl.ylines = False
    gl.xlocator = mticker.FixedLocator([-90, -135, 180, 135, 90])
    gl.ylocator = mticker.FixedLocator([-20, -10, 0, 10, 20])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.xlabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 12.5,'color': 'black', 'weight': 'bold'}
    ax.add_geometries([Ering], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth=2.5)
    ax.add_geometries([Wring], ccrs.PlateCarree(), facecolor='none', edgecolor='dimgrey', linewidth =2.5)
    ax.set_aspect('equal')
    plt.title(var_name[i])

In [ ]:
def glob_trend_plot(t, p):
    ax = fig.add_subplot(2,3,i+1, projection=ccrs.Robinson(central_longitude=180))
    ax.coastlines()
    plt.contourf(lon,lat,t[i],cmap='bwr', transform=ccrs.PlateCarree()) 
    plt.pcolor(lon,lat,t[i], cmap='bwr', vmin=-.001, vmax=.001, transform=ccrs.PlateCarree())    
    #plt.colorbar(orientation='horizontal')
    cs = ax.contourf(lon,lat, p[i], colors='none' ,hatches=["."],extend='lower', transform=ccrs.PlateCarree())
    ax.set_aspect('equal')
    plt.rcParams['font.size'] = '14'
    plt.title(var_name[i])

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(lt)):
    glob_trend_plot(lt, lt_sig)
plt.suptitle('dD Correlation with ' + region, y =.65)
#plt.rcParams['font.size'] = '24'
plt.subplots_adjust(hspace=-1, wspace=0)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.dpi = 600
for i in range(len(lt)):
    trop_plot_trend(lt, lt_sig)
plt.suptitle('dD Linear Trend with ' + region, y =.65)
#plt.rcParams['font.size'] = '24'
plt.subplots_adjust(hspace=-1, wspace=0)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
plt.tight_layout()
plt.savefig(variable + "glob_cor_post_anom.jpg")

# Time Series

In [ ]:
%%time
index_var = compute_index(anom_var18).mean(dim = 'ensemble')
index_var_ghg = compute_index(anom_var_ghg18).mean(dim = 'ensemble')
#index_lulc = compute_index(anom_index_lulc)
index_var_oza = compute_index(anom_var_oza18).mean(dim = 'ensemble')
index_var_cntl = compute_index(anom_var_cntl18)

In [ ]:
%%time
index_var = compute_index(anom_index).mean(dim = 'ensemble')
index_var_ghg = compute_index(anom_index_ghg).mean(dim = 'ensemble')
#index_lulc = compute_index(anom_index_lulc)
index_var_oza = compute_index(anom_index_oza).mean(dim = 'ensemble')
index_var_cntl = compute_index(anom_index)

In [ ]:
ts_list = [index_var, index_var_ghg, index_var_oza]

In [ ]:
dt = 1/12  # In years
t_oza = np.arange(0,1872)*dt +1850

In [ ]:
var_post = ['FULL', 'GHG','OZA'] # OZA Removed

In [ ]:
colors = ['green', 'blue','darkorange']
colors_line = ['green', 'blue','orange']
colors_ribbon = ['lightgreen', 'lightblue','lightyellow']
lent_post = len(ts_list)
variable = 'dD'

In [ ]:
def roll_post(r):
   # ax = fig.add_subplot(sharex=True, sharey=False)
    post = r[i].sel(time = slice('1850-01-01', '2005-12-01'))
    res = stats.linregress(t_oza,post)
    roll_avg = post.rolling(time=120, center=True).mean()
    ax.set_title(var_post[i])
    #ax.set_ylim([-.4,.4])
    plt.rcParams['font.size'] = '30'
    plt.plot(t_oza,roll_avg, color = colors[i], label = var_post[i])
    plt.plot(t_oza, (res.intercept + res.slope*t_oza), colors_line[i])
    plt.fill_between(t_oza, roll_avg + roll_avg.std(),roll_avg - roll_avg.std(),  edgecolor= colors_ribbon[i], facecolor=colors_ribbon[i], alpha=1)
    plt.ylabel('dD (per mille)', fontsize = '28')
    fig.suptitle('dD Difference East (210-270) - West (90-150) Anomalies')
    #print('Slope Pre' + var_post[i] + res.slope)
    #print('p-value' +var_post[i] + res.pvalue)

In [ ]:
# Full Time
fig = plt.figure(figsize=(40,20))
for i in range(lent_post):
    roll_post(ts_list)
    plt.ylim(-5,5)#
plt.xlabel('Year', fontsize = '30')
plt.rcParams['font.size'] = '30'
plt.rcParams['font.family']= 'sans-serif'
plt.suptitle(variable +' 10 Year Rolling Average ' + region , y = 0.9, fontsize = '35',fontweight="bold")
#plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.legend(mode = "expand", loc = 'lower center', bbox_to_anchor=(0.25,-0.25,0.5, -0.75), ncol =5)
plt.savefig(variable + "roll_post_mc.jpg")